In [2]:
#using Pkg; Pkg.activate("../../../")
using Revise, CausalityTools

  Activating project at `~/Code/Repos/Temp/CausalityTools.jl`


  ** incremental compilation may be fatally broken for this module **



In [61]:
using StaticArrays: MVector
"""
    Henon3() <: DiscreteDefinition
    Henon3(; a = 0.1, b = 0.3, c = 0.1, xi = [0.1, 0.2, 0.3])

The `Henon3` system is a lagged discrete dynamical system. The initial condition
`xi` is repeated over the three first time steps before iteration starts.
"""
struct Henon33333{T, S, A, B, C} <: DiscreteDefinition
    m₁::MVector{2, T} # holds past states of x1
    m₂::MVector{2, T} # holds past states of x2
    m₃::MVector{2, T} # holds past states of x3
    xi::S
    a::A
    b::B
    c::C

    function Henon33333(; a::A = 1.4, b::B = 0.3, c::C = 0.1, 
            xi::S = [0.4, 0.5, 0.6]) where {A, B, C, S}
        T = eltype(1.0)
        m₁ = MVector{2, T}(repeat([xi[1]], 2))      
        m₂ = MVector{2, T}(repeat([xi[2]], 2))        
        m₃ = MVector{2, T}(repeat([xi[3]], 2))        
        return new{T, S, A, B, C}(m₁, m₂, m₃, xi, a, b, c)
    end
end

function system(s::Henon33333)
    return DiscreteDynamicalSystem(eom_henon33333, s.xi, s)
end

function eom_henon33333(u, p::Henon33333, t)
    # `u` is simply ignored here, because the state is stored in the memory vectors
    m₁, m₂, m₃ = p.m₁, p.m₂, p.m₃
    x₁₁, x₁₂ = m₁[1], m₁[2]
    x₂₁, x₂₂ = m₂[1], m₂[2]
    x₃₁, x₃₂ = m₃[1], m₃[2]

    a, b, c = p.a, p.b, p.c
    dx₁= a - x₁₁^2 + b*x₁₂
    dx₂= a - c*x₁₁*x₂₁ - (1 - c)*x₂₁^2 + b*x₂₂
    dx₃= a - c*x₂₁*x₃₁ - (1 - c)*x₃₁^2 + b*x₃₂

    new_state = SVector{3}(dx₁, dx₂, dx₃)
    update_state!(p, new_state) # Update memory vectors
    return new_state
end

function update_state!(p::Henon33333, xnew::SVector{3})
    p.m₁[2] = p.m₁[1]
    p.m₁[1] = xnew[1]
    p.m₂[2] = p.m₂[1]
    p.m₂[1] = xnew[2]
    p.m₃[2] = p.m₃[1]
    p.m₃[1] = xnew[3]
end


update_state! (generic function with 2 methods)

In [62]:
sys = system(Henon33333());

3-dimensional discrete dynamical system
 state:       [0.4, 0.5, 0.6]
 rule f:      eom_henon33333
 in-place?    false
 jacobian:    ForwardDiff
 parameters:  Henon33333{Float64, Vector{Float64}, Float64, Float64, Float64}([-0.3296, 1.36], [-0.160202, 1.305], [0.0672386, 1.226], [0.4, 0.5, 0.6], 1.4, 0.3, 0.1)

In [66]:
trajectory(sys, 1000)

3-dimensional Dataset{Float64} with 1001 points
  0.4        0.5        0.6
  1.62032    0.733367   1.61086
 -1.12199    1.06447   -1.22586
  0.627231   0.719658   0.661283
  0.669983   1.20809    0.591086
  1.13929    0.221434   1.21253
  0.30301    1.69307    0.227264
  1.64997   -1.1647     1.6788
 -1.23151    0.879215  -0.872819
  0.378384   0.463148   1.29475
  ⋮                    
  0.781281   0.533008   1.78612
  1.07028    1.31244   -1.53514
  0.488876  -0.130807   0.0163228
  1.48209    1.78473    0.939432
 -0.649913  -1.77048    0.442955
  1.42224   -1.00078    1.58367
 -0.817736   0.109789  -0.565819
  1.15798    1.0979     1.59318
 -0.186237   0.220965  -1.22905